Set configuration for this measurement

In [25]:
config_path = r'resonator_PD.toml'


import tomlkit

# Assuming 'config.toml' is your file
with open(config_path, 'r') as file:
    content = file.read()
    resPD_config = tomlkit.parse(content)



Load packages

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime
import tomli_w
import copy

config parser

In [27]:


data_output_folder = resPD_config["output"]["data_path"]


new_config = {
    "hardware":resPD_config["hardware"],
    "sample":resPD_config["sample"],
}
measurement = []

attenuation = resPD_config["hardware"]["attenuation"]
for measurement_info in resPD_config["resonator"]:
    # print(measurement_info)
    single_resonator = {
        "label": measurement_info["label"],
        "frequency": measurement_info["frequency"],
        "IF_bandwidth":measurement_info["IF_bandwidth"]
    }

    vna_power_setting = measurement_info["power"]
    # print(vna_power_setting)
    if "custom" not in vna_power_setting.keys():
        print("linear spacing power")
        min_power = vna_power_setting["min"] 
        max_power = vna_power_setting["max"] 
        step = vna_power_setting["step"] 
        print(min_power,max_power,step)
        power_range = np.arange( min_power, max_power+step/2, step)
    else:
        power_range = np.array(vna_power_setting["custom"])
        min_power = np.min(power_range)

    power_range = power_range 

    IF_bandwidth = measurement_info["frequency"]["points"]


    max_repeat = int(measurement_info["snr"]["max_repeat"])
    min_repeat = int(measurement_info["snr"]["min_repeat"])


    for p in power_range:
        r_sn = measurement_info["snr"]["power_ave_ratio"]
        repeat = 10**((r_sn -(p-attenuation) )/10)
        if repeat < min_repeat: repeat = min_repeat
        if repeat > max_repeat: repeat = max_repeat

        single_power = copy.deepcopy(single_resonator)
        single_power["power"] = p
        single_power["repeat"] = int(repeat)
        single_power["output"] = f"{data_output_folder}\\{measurement_info['label']}\\att{attenuation}_{str(p)}"

        measurement.append(single_power)

new_config["measurement"] = measurement
# Export to a TOML file
with open(f'power_dep_resonator.toml', 'wb') as file:
    tomli_w.dump(new_config, file)

linear spacing power
-50 0 20
linear spacing power
-50 0 20


Create task

Connect with VNA

In [28]:

from driver import get_VNA

# vna = get_VNA(vna_address,vna_model)

# vna.check_error()




Measurement

Plot